# Fabric

1. Crea un fichero llamado `fabfile.py` con el siguiente contenido

```python
from fabric import Connection, task

@task
def development(ctx):
    ctx.user = 'vagrant'
    ctx.host = '192.168.33.10'
    ctx.connect_kwargs = {"password": "vagrant"}

@task
def deploy(ctx):
    with Connection(ctx.host, ctx.user, connect_kwargs=ctx.connect_kwargs) as conn:
        conn.run("uname")
        conn.run("ls")
```

2. Activa el entorno virtual y ejecuta el siguiente comando de Fabric en la consola:

```
source .venv/bing/activate
fab vagrant_env deploy
```  
Output:
```
Linux
my-first-blog
my-flask-project
```


# Despliegue de una aplicación Django con Fabric

A continuacion vamos a desplegar una aplicación Django con Fabric en el servidor de vagrant

---------------
1. Vamos a añadir algunos import más que nos serán útiles a la hora de definir las siguientes tareas:

```
import sys
import os

from fabric import Connection, task
```

-------------------
2. Vamos a definir una serie de variables que nos servirán como constantes en el despliegue de la aplicación:

```python
PROJECT_NAME = "my-first-blog"
PROJECT_PATH = f"~/{PROJECT_NAME}"
REPO_URL = "https://github.com/aliciapj/my-first-blog.git"
VENV_PYTHON = f'{PROJECT_PATH}/.venv/bin/python'
VENV_PIP = f'{PROJECT_PATH}/.venv/bin/pip'
```


------------------
3. Ahora deberíamos tener el fichero con algo como esto:
    
```python
import sys
import os
from fabric import Connection, task

PROJECT_NAME = "my-first-blog"
PROJECT_PATH = f"~/{PROJECT_NAME}"
REPO_URL = "https://github.com/aliciapj/my-first-blog.git"
VENV_PYTHON = f'{PROJECT_PATH}/.venv/bin/python'
VENV_PIP = f'{PROJECT_PATH}/.venv/bin/pip'

@task
def development(ctx):
    ctx.user = 'vagrant'
    ctx.host = '192.168.33.10'
    ctx.connect_kwargs = {"password": "vagrant"}


@task
def deploy(ctx):
    with Connection(ctx.host, ctx.user, connect_kwargs=ctx.connect_kwargs) as conn:
        conn.run("uname")
        conn.run("ls")
```

--------------
4. Cambiamos el método de deploy para hacer un git clone del repositorio de nuestro proyecto django en el servidor remoto. Vamos a extraer del deploy la funcionalidad de crear la conexión para poder reutilizarla en otras tareas. Como va a ser un método auxiliar, no hace falta ponerle el decorador de `@task`:

```python
import sys
import os
from fabric import Connection, task

PROJECT_NAME = "my-first-blog"
PROJECT_PATH = f"~/{PROJECT_NAME}"
REPO_URL = "https://github.com/aliciapj/my-first-blog.git"
VENV_PYTHON = f'{PROJECT_PATH}/.venv/bin/python'
VENV_PIP = f'{PROJECT_PATH}/.venv/bin/pip'

@task
def development(ctx):
    ctx.user = 'vagrant'
    ctx.host = '192.168.33.10'
    ctx.connect_kwargs = {"password": "vagrant"}

def get_connection(ctx):
    try:
        with Connection(ctx.host, ctx.user, connect_kwargs=ctx.connect_kwargs) as conn:
            return conn
    except Exception as e:
        return None

@task
def deploy(ctx):
    conn = get_connection(ctx)
    if conn is None:
        sys.exit("Failed to get connection")
```

> Prueba la tarea con `fab development deploy` y comprueba que no aparezcan errores

---------------------
5. A partir de ahora vamos a añadir tareas a nuestro proceso de deploy, definiendo una nueva `@task` y añadiéndola al método de deploy. Empezaremos por la de hacer un `git clone` del repo que hemos definido en la constante `REPO_URL`.  
En todas las tareas empezaremos obteniendo la conexión por parámetro y a continuación, ejecutando los comandos correspondientes.

```python
@task
def clone(ctx): 
    print(f"clone repo {REPO_URL}...")   
    
    if isinstance(ctx, Connection):
        conn = ctx
    else:
        conn = get_connection(ctx)

    # obtengo las carpetas del directorio
    ls_result = conn.run("ls").stdout

    # divido el resultado para tener cada carpeta en un objeto de una lista
    ls_result = ls_result.split("\n")

    # si el nombre del proyecto ya está en la lista de carpetas
    # no es necesario hacer el clone 
    if PROJECT_NAME in ls_result:
        print("project already exists")
    else:
        conn.run(f"git clone {REPO_URL} {PROJECT_NAME}")
```

    Y ahora añadimos la tarea al proceso de deploy:
    
```python
@task
def deploy(ctx):
    conn = get_connection(ctx)
    if conn is None:
        sys.exit("Failed to get connection")
    
    clone(conn)
```

> Prueba la tarea con `fab development deploy`, comprueba que el resultado es algo como lo siguiente y comprueba que la operación se ha ejecutado correctamente en el servidor  
```
clone repo https://github.com/aliciapj/my-first-blog.git...
my-flask-project
Cloning into 'my-first-blog'...
```

-----------------------------
6. Vamos a hacer un checkout de la rama `main` del repositorio:

```python
@task
def checkout(ctx, branch=None):
    print(f"checkout to branch {branch}...")

    if branch is None:
        sys.exit("branch name is not specified")
    
    if isinstance(ctx, Connection):
        conn = ctx
    else:
        conn = get_connection(ctx)
    
    with conn.cd(PROJECT_PATH):
        conn.run(f"git checkout {branch}")
```

Y añade la nueva tarea a la función de deploy

```python
@task
def deploy(ctx):
    conn = get_connection(ctx)
    if conn is None:
        sys.exit("Failed to get connection")
    
    clone(conn)
    checkout(conn, branch="main")
```

> Prueba la tarea con `fab development deploy`, comprueba que el resultado es algo como lo siguiente y comprueba que la operación se ha ejecutado correctamente en el servidor  
```
clone repo https://github.com/aliciapj/my-first-blog.git...
my-first-blog
my-flask-project
project already exists
checkout to branch main...
Already on 'main'
Your branch is up to date with 'origin/main'.
```

----------------
7. Crea una tarea para hacer un git pull de la rama:

```python
@task
def pull(ctx, branch="main"):

    print(f"pulling latest code from {branch} branch...")

    if branch is None:
        sys.exit("branch name is not specified")

    if isinstance(ctx, Connection):
        conn = ctx
    else:
        conn = get_connection(ctx)

    with conn.cd(PROJECT_PATH):
        conn.run(f"git pull origin {branch}")
```

Y añade la nueva tarea a la función de deploy

```python
@task
def deploy(ctx):
    conn = get_connection(ctx)
    if conn is None:
        sys.exit("Failed to get connection")

    clone(conn)
    checkout(conn, branch="main")
    pull(conn, branch="main")
```

> Prueba la tarea con `fab development deploy`, comprueba que el resultado es algo como lo siguiente y comprueba que la operación se ha ejecutado correctamente en el servidor  

```
clone repo https://github.com/aliciapj/my-first-blog.git...
my-first-blog
my-flask-project
project already exists
checkout to branch main...
Your branch is up to date with 'origin/main'.
Already on 'main'
pulling latest code from main branch...
From https://github.com/aliciapj/my-first-blog
 * branch            main     -> FETCH_HEAD
Already up to date.
```

------------------
8. Crea una tarea para crear un virtual environment con Python 3 (recuerda que el comando en linux para un entorno llamado `.venv` es `python3 -m venv .venv`.  
Además, instala a través del pip del virtualenv creado la librería de django

```python

@task
def create_venv(ctx):
    
    print("creating venv....")
    
    if isinstance(ctx, Connection):
        conn = ctx
    else:
        conn = get_connection(ctx)
    with conn.cd(PROJECT_PATH):
        conn.run("python3 -m venv .venv")
        conn.run(f"{VENV_PIP} install django")
```

Y añade la nueva tarea a la función de deploy

```python
@task
def deploy(ctx):
    conn = get_connection(ctx)
    if conn is None:
        sys.exit("Failed to get connection")

    clone(conn)
    checkout(conn, branch="main")
    pull(conn, branch="main")
    create_venv(conn)
```

> Prueba la tarea con `fab development deploy`, comprueba que el resultado es algo como lo siguiente y comprueba que la operación se ha ejecutado correctamente en el servidor  

```
clone repo https://github.com/aliciapj/my-first-blog.git...
my-first-blog
my-flask-project
project already exists
checkout to branch main...
Already on 'main'
Your branch is up to date with 'origin/main'.
pulling latest code from main branch...
From https://github.com/aliciapj/my-first-blog
 * branch            main     -> FETCH_HEAD
Already up to date.
creating venv....
Collecting django
  Using cached Django-3.2-py3-none-any.whl (7.9 MB)
Collecting sqlparse>=0.2.2
  Using cached sqlparse-0.4.1-py3-none-any.whl (42 kB)
Collecting asgiref<4,>=3.3.2
  Using cached asgiref-3.3.4-py3-none-any.whl (22 kB)
Collecting pytz
  Using cached pytz-2021.1-py2.py3-none-any.whl (510 kB)
Installing collected packages: sqlparse, asgiref, pytz, django
Successfully installed asgiref-3.3.4 django-3.2 pytz-2021.1 sqlparse-0.4.1
```

--------------
9. Crea una tarea para ejecutar las migraciones de django

```python
@task
def migrate(ctx):
    print("checking for django db migrations...")

    if isinstance(ctx, Connection):
        conn = ctx
    else:
        conn = get_connection(ctx)
        
    with conn.cd(PROJECT_PATH):
        conn.run(f"{VENV_PYTHON} manage.py migrate")
```

Y añade la nueva tarea a la función de deploy

```python
@task
def deploy(ctx):
    conn = get_connection(ctx)
    if conn is None:
        sys.exit("Failed to get connection")

    clone(conn)
    checkout(conn, branch="main")
    pull(conn, branch="main")
    create_venv(conn)
    migrate(conn)
```

> Prueba la tarea con `fab development deploy`, comprueba que el resultado es algo como lo siguiente y comprueba que la operación se ha ejecutado correctamente en el servidor  

```
clone repo https://github.com/aliciapj/my-first-blog.git...
my-first-blog
my-flask-project
project already exists
checkout to branch main...
Already on 'main'
Your branch is up to date with 'origin/main'.
pulling latest code from main branch...
From https://github.com/aliciapj/my-first-blog
 * branch            main     -> FETCH_HEAD
Already up to date.
creating venv....
Requirement already satisfied: django in ./.venv/lib/python3.8/site-packages (3.2)
Requirement already satisfied: pytz in ./.venv/lib/python3.8/site-packages (from django) (2021.1)
Requirement already satisfied: asgiref<4,>=3.3.2 in ./.venv/lib/python3.8/site-packages (from django) (3.3.4)
Requirement already satisfied: sqlparse>=0.2.2 in ./.venv/lib/python3.8/site-packages (from django) (0.4.1)
checking for django db migrations...
Operations to perform:
  Apply all migrations: admin, auth, blog, contenttypes, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying admin.0003_logentry_add_action_flag_choices... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying auth.0009_alter_user_last_name_max_length... OK
  Applying auth.0010_alter_group_name_max_length... OK
  Applying auth.0011_update_proxy_permissions... OK
  Applying auth.0012_alter_user_first_name_max_length... OK
  Applying blog.0001_initial... OK
  Applying sessions.0001_initial... OK
System check identified some issues:

WARNINGS:
blog.Post: (models.W042) Auto-created primary key used when not defining a primary key type, by default 'django.db.models.AutoField'.
        HINT: Configure the DEFAULT_AUTO_FIELD setting or the BlogConfig.default_auto_field attribute to point to a subclass of AutoField, e.g. 'django.db.models.BigAutoField'.
```

### Con esto nuestra aplicación Django ya estaría lista para arrancar
--------------

# Despliegue en varios entornos

Si tuviera dos entornos de ejecución (por ejemplo, `development` y `production`), con los siguientes parámetros distintos:

```
development_settings = {
    username = 'dev_user',
    password = 'dev_pass',
    github_branch = 'development',
}

production_settings = {
    username = 'pro_user',
    password = 'pro_pass',
    github_branch = 'production',
}
```

¿Cómo tendría que adaptar mi `fabfile.py` para que pueda desplegar en un entorno o en el otro con los siguientes comandos?
```
fab development deploy
fab production deploy
```

### Pistas:
- Crea 2 task para establecer las variables de contexto (`ctx`) y el resto de variables, ponlas como globales al principio y establece su valor en estos métodos
- Puedes guardar variables adicionales a las de la conexión (por ejemplo, la rama de github) en variables de entorno del usuario a través de `ctx.config.run.env`
- Crea una task para comprobar si el usuario existe en el servidor, y si no existe, crearlo a través del usuario vagrant
- Conéctate a través del usuario del entorno para realizar el resto de tareas